In [1]:
import sys
sys.path.append("../..")
from sources import *
import os
import os.path
from IPython.display import clear_output
import time
import geo.countries
from geo.utils import extract_countries
from data.Frame import Frame
from geo.countries import Countries
import requests
import urllib
import json
import requests
import json

In [2]:
c = Countries()
c.load_wb()

In [ ]:
from pymongo import MongoClient
import db
client = MongoClient('stats.whereistomas.org', username='root', password='Root!0676')
db = client["stats"]
url = 'http://country.io/names.json'
response = urllib.request.urlopen(url)
data = response.read().decode('utf-8')
country_names = json.loads(data)

In [ ]:
cache_folder = '..'
# 604800 is on week
max_age = 604800

def get_file_path(name):
    return str(cache_folder) + '/' + str(name) + '.json'

def check_age(name):
    if (os.path.getmtime(get_file_path(name)) - time.time()) < -max_age:
        print("Updating " + str(name) + " from server.")
        retrieve_and_cache(name)
        
def check_cache(name):
    return (os.path.exists(name))

def write_cache(data, file_path):
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile)
        outfile.close()
        
def load_cache(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return (data)

def retrieve_and_cache(name):
    url = 'https://raw.githubusercontent.com/iancoleman/cia_world_factbook_api/master/data/factbook.json'
    r = requests.get(url)
    f = get_file_path(name)
    write_cache(r.json(), f)

def get_CIA_factbook(**kwargs):
    p = get_file_path(kwargs['name'])
    if check_cache(p) == False:
        retrieve_and_cache(name=kwargs['name'])
    else:
        check_age(kwargs['name'])
    cached_data = load_cache(p)
    
    return cached_data

def add_to_profile(name, kind, config, profile):
    profile.append({'name':name, 'kind': kind, 'config':config})
    return profile

def process_wb_data(c):
    new_g = []            
    for g in c['groups']:
        new_g.append(g.lower())    
    
    c.groups = new_g
        
    return new_g

fd = get_CIA_factbook(name='factbook.json')

In [ ]:
def process_profile_data(iso2code):
    profile = []
    
    country_name = country_names[iso2code].lower().replace(" ", "_")
    
    if (iso2code == 'BS'):
        country_name = "bahamas_the"
    
    if (iso2code == 'CI'):
        country_name = "cote_d'_ivoire"
        
    if (iso2code == 'CD'):
        country_name = "congo_democratic_republic_of_the"
        
    if (iso2code == 'CG'):
        country_name = "congo_republic_of_the"
        
    if (iso2code == 'CV'):
        country_name = "cabo_verde"
        
    if (iso2code == 'CZ'):
        country_name = "czechia"
        
    if (iso2code == 'FM'):
        country_name = "micronesia_federated_states_of"
        
    if (iso2code == 'GM'):
        country_name = "gambia_the"
        
    if (iso2code == 'GW'):
        country_name = "guinea_bissau"
    
    if (iso2code == 'KR'):
        country_name = "korea_south"
        
    if (iso2code == 'MO'):
        country_name = "macau"
        
    if (iso2code == 'MK'):
        country_name = "north_macedonia"
    
    if (iso2code == 'MM'):
        country_name = "burma"
    
    if (iso2code == 'KP'):
        country_name = "korea_north"
    
    if (iso2code == 'SZ'):
        country_name = "eswatini"
        
    if (iso2code == 'TL'):
        country_name = "timor_leste"
        
    if (iso2code == 'VI'):
        country_name = 'virgin_islands'
    
    profile = add_to_profile('paragraph', 'text', {'kind': 'p', 'className': 'country-introductory-text', 'text': fd['countries'][country_name]['data']['introduction']['background']}, profile)
    
    profile = add_to_profile('paragraph', 'text', {'kind': 'p', 'className': 'country-introductory-text', 'text': "Flag: " + fd['countries'][country_name]['data']['government']['flag_description']['description']}, profile)
    
    profile = add_to_profile('header', 'text', {'kind': 'h3', 'text':'Population'}, profile)
    
    profile = add_to_profile('SPPOPTOTL', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2000,2018))], 
                                                        'width':1000, 'height':400}, profile)
    
    profile = add_to_profile('header', 'text', {'kind': 'h3', 'text':'Macrodynamics'}, profile)
    
    profile = add_to_profile('NYGDPMKTPCD', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    
    profile = add_to_profile('NYGDPTOTLRTZS', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    
    profile = add_to_profile('NYGDPMKTPKDZG', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    
    
    profile = add_to_profile('header', 'text', {'kind': 'h3', 'text':'Environment'}, profile)
    
    profile = add_to_profile('paragraph', 'text', {'kind': 'p', 'className': 'country-introductory-text', 'text': "Meteorology: " + fd['countries'][country_name]['data']['geography']['climate']}, profile)
    
    profile = add_to_profile('ENATMCO2EPC', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    
    profile = add_to_profile('header', 'text', {'kind': 'h3', 'text':'Human Development'}, profile)
    
    profile = add_to_profile('NYGNPPCAPCD', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    profile = add_to_profile('SEPRMUNERFE', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    
    profile = add_to_profile('SPDYNLE00FEIN', 'indicator', {'kind':'linechart', 
                                                        'years':[str(year) for year in list(range(2011,2018))], 
                                                        'width':400, 'height':400}, profile)
    
        
        
    return profile

In [ ]:
db["countries"].drop()
for ct in c:
    try:
        if (ct['capital_longitude'] != ""):
            g = process_wb_data(ct)
            profile =  process_profile_data(ct["iso2code"])       
            db["countries"].update_one({ "name" : ct['name'].lower()}, {"$set":{"name": country_names[ct["iso2code"]].lower(),
                                                                       "aliases": ct["aliases"],
                                                                       "iso2code": ct["iso2code"],
                                                                       "iso3code": ct["iso3code"],
                                                                       "income_level": ct["incomelevel"],
                                                                       "capital_longitude": ct["capital_longitude"],
                                                                       "capital_latitude": ct["capital_latitude"],
                                                                       "lendingtype": ct["lendingtype"],
                                                                       "region": ct["region"],
                                                                       "groups": g,
                                                                       "profile": profile}}, True)
    except ValueError:
        pass